# Build CNN For Comparision 

In [ ]:
#Copyright 2019, Seokjun Bu, Softcomputing LAB all rights reserved.
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, Reshape, Conv2D, Dense, MaxPool2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = np.load('/content/drive/My Drive/test_colab/dataset_2_char_cat_45000_15000_140.npy')

In [4]:
# Train, Test Split
X, Y = data[:, 1:], data[:, 0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=11)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(42000, 14000) (42000,)
(18000, 14000) (18000,)


In [ ]:
Y_train_cnn, Y_test_cnn = to_categorical(Y_train), to_categorical(Y_test)

In [11]:
cnn_input = Input((X_train.shape[1],))
H = Reshape((100, 140, 1))(cnn_input)

H = Conv2D(filters=32, kernel_size=(2,2), padding="same", activation='relu')(H)
H = MaxPool2D((2, 2))(H)

H = Conv2D(filters=64, kernel_size=(2,2), padding="same", activation='relu')(H)
H = MaxPool2D((2, 2))(H)

H = Flatten()(H)
H = Dense(4, activation='relu')(H)

cnn_output = Dense(2, activation = 'softmax')(H)
cnn_model = Model(cnn_input, cnn_output)
cnn_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 14000)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 100, 140, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 140, 32)      160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 70, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 70, 64)        8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 35, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 56000)             0   

In [16]:
%%time
adam = Adam(lr = 1e-04)
cnn_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='auto', patience=10, restore_best_weights=True)
history = cnn_model.fit(X_train, Y_train_cnn, validation_split=0.2, 
                        epochs=500, batch_size=512, shuffle=True, verbose=1, callbacks=[es])

Train on 33600 samples, validate on 8400 samples
Epoch 1/500
33600/33600 [==============================] - 9s 272us/step - loss: 0.4433 - acc: 0.8131 - val_loss: 0.3628 - val_acc: 0.8424
Epoch 2/500
33600/33600 [==============================] - 8s 240us/step - loss: 0.3214 - acc: 0.8605 - val_loss: 0.2822 - val_acc: 0.8806
Epoch 3/500
33600/33600 [==============================] - 8s 239us/step - loss: 0.2658 - acc: 0.8851 - val_loss: 0.2506 - val_acc: 0.8956
Epoch 4/500
33600/33600 [==============================] - 8s 239us/step - loss: 0.2410 - acc: 0.8975 - val_loss: 0.2330 - val_acc: 0.9037
Epoch 5/500
33600/33600 [==============================] - 8s 239us/step - loss: 0.2248 - acc: 0.9059 - val_loss: 0.2240 - val_acc: 0.9075
Epoch 6/500
33600/33600 [==============================] - 8s 240us/step - loss: 0.2129 - acc: 0.9118 - val_loss: 0.2122 - val_acc: 0.9139
Epoch 7/500
33600/33600 [==============================] - 8s 239us/step - loss: 0.2041 - acc: 0.9166 - val_loss: 0.2

In [17]:
y_pred_model = cnn_model.predict(X_test.astype("float"))
y_pred = np.where(y_pred_model > 0.5, 1, 0)
print(classification_report(Y_test_cnn, y_pred, target_names=['class 0', 'class 1']))

              precision    recall  f1-score   support

     class 0       0.95      0.98      0.96     13546
     class 1       0.93      0.84      0.88      4454

   micro avg       0.94      0.94      0.94     18000
   macro avg       0.94      0.91      0.92     18000
weighted avg       0.94      0.94      0.94     18000
 samples avg       0.94      0.94      0.94     18000



In [ ]:
def func2():
  cnn_input = Input((X_train.shape[1],))
  H = Reshape((100, 140, 1))(cnn_input)

  H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
  H = MaxPool2D((2,2))(H)

  H = Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu')(H)
  H = MaxPool2D((2,2))(H)
  
  H = Flatten()(H)
  H = Dense(4, activation = 'relu')(H)

  cnn_output = Dense(2, activation = 'softmax')(H)

  cnn_model = Model(cnn_input,cnn_output)
  
  cnn_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  
  return cnn_model

In [ ]:
cnn = KerasClassifier(build_fn=func2, epochs=10, batch_size=10, verbose=0)
recall_cnn = cross_val_score(cnn, X_train, Y_train, cv=10, scoring='recall')

In [25]:
recall_cnn

array([0.87546816, 0.82075472, 0.91468254, 0.89468691, 0.89258555,
       0.86363636, 0.        , 0.89126394, 0.89878163, 0.89514563])